In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys

from customerSeg import data_dir, data_path

ImportError: cannot import name 'data_dir' from 'customerSeg' (/Users/jared/DevProjects/customer-segmenter/customerSeg/customerSeg/__init__.py)

In [4]:
data_path

PosixPath('/Users/jared/DevProjects/customer-segmenter/data/Online Retail.xlsx')

In [6]:
df = pd.read_excel(data_path, engine='openpyxl')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [18]:
# df.to_parquet('/Users/jared/DevProjects/customer-segmenter/data/Online Retail',
#              engine='fastparquet')

In [20]:
from pyspark import SparkContext

In [24]:
sc = SparkContext('local', '1_notebook')

Exception: Java gateway process exited before sending its port number